In [ ]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.1.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:2 http://security.ubuntu.com/ubuntu bionic-security InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:10 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:13 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:15 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic 

In [ ]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-10-09 22:57:22--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.1’

postgresql-42.2.16. 100%[===================>] 979.38K  4.95MB/s    in 0.2s    

2021-10-09 22:57:22 (4.95 MB/s) - ‘postgresql-42.2.16.jar.1’ saved [1002883/1002883]



In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [ ]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Major_Appliances_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Major_Appliances_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   16199106|R203HPW78Z7N4K|B0067WNSZY|     633038551|FGGF3032MW Galler...|Major Appliances|          5|            0|          0|   N|                Y|If you need a new...|What a great stov...| 2015-08-31|
|         US|   16374060|R2EAIGVLEALSP3|B002QSXK60|     811766671|Best Hand Clothes...|Major Appliances|          5|    

In [ ]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R203HPW78Z7N4K|          5|            0|          0|   N|                Y|
|R2EAIGVLEALSP3|          5|            1|          1|   N|                Y|
|R1K1CD73HHLILA|          5|            0|          0|   N|                Y|
|R2KZBMOFRMYOPO|          5|            1|          1|   N|                Y|
| R6BIZOZY6UD01|          5|            0|          0|   N|                Y|
|R1MCXZFNF8E7Y0|          1|            0|          0|   N|                Y|
|R3EMB3E3ODR6BW|          5|            2|          2|   N|                Y|
| RJTONVTTOPJ5S|          5|            0|          0|   N|                Y|
|R21U5QZ2CQECUM|          4|            0|          0|   N|                Y|
| RL2BBC51H89DH|          4|            0|          0|   N|     

In [ ]:
# 1.
# Filter the data and create a new DataFrame or table to retrieve all the rows where
# the total_votes count is equal to or greater than 20 to pick reviews that are
# more likely to be helpful and to avoid having division by zero errors later on.
from pyspark.sql.functions import col, count, round
votes20_df = vine_df.filter(col("total_votes") >= 20)
votes20_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1QXNQBTH7LIUB|          3|            9|         22|   N|                Y|
|R3BC75VKJK6LNV|          4|           83|         89|   N|                Y|
| R5XKK92G5N9FU|          1|           18|         22|   N|                N|
| RYMOWYW38WKOB|          5|           35|         35|   N|                N|
| RYHZAVESD0T37|          1|           39|         45|   N|                Y|
|R19Y5VBOF3BQOG|          1|           26|         35|   N|                Y|
| R8V9F3139Z1WZ|          1|           30|         34|   N|                N|
|R3RNY5OLY451GS|          1|           25|         30|   N|                Y|
|R16LPVYSJOAH1T|          1|           20|         20|   N|                N|
|R2NLL7ISMM5QYQ|          4|           89|         94|   N|     

In [ ]:
# 2.
# Filter the new DataFrame or table created in Step 1 and create a new DataFrame
# or table to retrieve all the rows where the number of helpful_votes divided by
# total_votes is equal to or greater than 50%.

x_df = votes20_df.withColumn("calc", (((votes20_df['helpful_votes'])/(votes20_df['total_votes']))*100))
x_df = x_df.filter(col("calc")>="50")
x_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|             calc|
+--------------+-----------+-------------+-----------+----+-----------------+-----------------+
|R3BC75VKJK6LNV|          4|           83|         89|   N|                Y|93.25842696629213|
| R5XKK92G5N9FU|          1|           18|         22|   N|                N|81.81818181818183|
| RYMOWYW38WKOB|          5|           35|         35|   N|                N|            100.0|
| RYHZAVESD0T37|          1|           39|         45|   N|                Y|86.66666666666667|
|R19Y5VBOF3BQOG|          1|           26|         35|   N|                Y|74.28571428571429|
| R8V9F3139Z1WZ|          1|           30|         34|   N|                N|88.23529411764706|
|R3RNY5OLY451GS|          1|           25|         30|   N|                Y|83.33333333333334|
|R16LPVYSJOAH1T|          1|           2

In [ ]:
# 3.
# Filter the DataFrame or table created in Step 2, and create a new DataFrame
# or table that retrieves all the rows where a review was written as part of 
# the Vine program (paid), vine == 'Y'.

vine_paid_df = x_df.filter(col("vine")=="Y")
vine_paid = vine_paid_df.count()
print(vine_paid)



35


In [ ]:
# 4.
# Repeat Step 3, but this time retrieve all the rows where the review was not
# part of the Vine program (unpaid), vine == 'N'.

vine_unpaid_df = x_df.filter(col("vine") == "N")
vine_unpaid = vine_unpaid_df.count()
print(vine_unpaid)



4957


In [ ]:
# 5.
# Determine the total number of reviews, the number of 5-star reviews, and the
# percentage of 5-star reviews for the two types of review (paid vs unpaid).

# z = total number of reviews
z = vine_df.count()
print("The total number of reviews is " + str(z) + ".")

z5 = vine_df.filter(col("star_rating")==5).count()
print("The number of 5-star rating is " + str(z5) + ".")
print("")

# vine_paid = Yes = y 
# 5-star paids = a
# vine_paid_pct5 = percent of Y's that are 5-stars

print("The number of Vine (paid) reviews is " + str(vine_paid) + ".")
a = vine_paid_df.filter(col("star_rating")==5).count()
print("The number of Vine (paid) 5-star ratings is " + str(a) + ".")
vine_paid_pct5 = (a/vine_paid)*100
print("The percent of Vine (paid) 5-star reviews is " + str(vine_paid_pct5) + "%.")
print("")

# vine_unpaid = No = n
# 5-star unpaid = b
# vine_unpaid_pct5 = percent of N's that are 5-stars

print("The number of non-Vine (unpaid) reviews is " + str(vine_unpaid) + ".")
b = vine_unpaid_df.filter(col("star_rating")==5).count()
print("The number of non-Vine (unpaid) 5-star ratings is " + str(b) + ".")
vine_unpaid_pct5 = (b/vine_unpaid)*100
print("The percent of unpaid 5-star reviews is " + str(vine_unpaid_pct5) + "%.")


The total number of reviews is 96901.
The number of 5-star rating is 49704.

The number of Vine (paid) reviews is 35.
The number of Vine (paid) 5-star ratings is 18.
The percent of Vine (paid) 5-star reviews is 51.42857142857142%.

The number of non-Vine (unpaid) reviews is 4957.
The number of non-Vine (unpaid) 5-star ratings is 1963.
The percent of unpaid 5-star reviews is 39.60056485777688%.
